# Credit Card Fraud Detection::

Download dataset from this link:

https://www.kaggle.com/mlg-ulb/creditcardfraud

The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 


### Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

# WORKFLOW :

1.Load Data

2.Check Missing Values ( If Exist ; Fill each record with mean of its feature )

3.Standardized the Input Variables. 

4.Split into 50% Training(Samples,Labels) , 30% Test(Samples,Labels) and 20% Validation Data(Samples,Labels).

5.Model : input Layer (No. of features ), 3 hidden layers including 10,8,6 unit & Output Layer with activation function relu/tanh (check by experiment).

6.Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)

7.Train the Model with Epochs (100).

8.If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .

9.Prediction should be > 92%
10.Evaluation Step
11Prediction


# Task::

## Identify fraudulent credit card transactions.

In [47]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [48]:
df = pd.read_csv('datasets/credit card/creditcard.csv')

In [49]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [51]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [52]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [53]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,...,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [54]:
np.where(df.applymap(lambda x: x==''))

(array([], dtype=int64), array([], dtype=int64))

In [55]:
(df['Amount'].values == 0).sum()

1825

In [56]:
non_fraud=df[df["Class"]==0]
fraud = df[df["Class"]==1]
non_fraud=non_fraud.sample(3*fraud.shape[0])
data = fraud.append(non_fraud, ignore_index=True)
x_data= data.drop(columns="Class", axis=0)
label = data["Class"]

In [57]:
non_fraud.shape

(1476, 31)

In [58]:
data.shape

(1968, 31)

In [59]:
train_data, test_data, train_labels, test_labels=train_test_split(x_data, label, test_size=0.3, random_state=1, stratify = label)

In [60]:
train_mean = train_data.mean(axis=0) # taking the mean of 
train_data -= train_mean
train_std = train_data.std(axis=0)
train_data /= train_std
test_data -= train_mean
test_data /= train_std

# Building the network

In [61]:
model = models.Sequential()
model.add(layers.Dense(20, activation='relu',kernel_regularizer=regularizers.l2(0.001), input_shape=(train_data.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(6, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [62]:
model.compile(loss= 'binary_crossentropy', optimizer= 'rmsprop', metrics=['accuracy'])

In [63]:
model.fit(train_data, train_labels, epochs=100,validation_split = 0.3)

Epoch 1/100
31/31 [==============================] - 2s 10ms/step - loss: 0.8280 - accuracy: 0.6359 - val_loss: 0.7033 - val_accuracy: 0.7657
Epoch 2/100
31/31 [==============================] - 0s 3ms/step - loss: 0.7893 - accuracy: 0.6924 - val_loss: 0.6789 - val_accuracy: 0.7391
Epoch 3/100
31/31 [==============================] - 0s 2ms/step - loss: 0.7703 - accuracy: 0.7210 - val_loss: 0.6671 - val_accuracy: 0.7391
Epoch 4/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6940 - accuracy: 0.7199 - val_loss: 0.6519 - val_accuracy: 0.7391
Epoch 5/100
31/31 [==============================] - 0s 2ms/step - loss: 0.7017 - accuracy: 0.7408 - val_loss: 0.6376 - val_accuracy: 0.7391
Epoch 6/100
31/31 [==============================] - 0s 2ms/step - loss: 0.6969 - accuracy: 0.7069 - val_loss: 0.6246 - val_accuracy: 0.7391
Epoch 7/100
31/31 [==============================] - 0s 3ms/step - loss: 0.6402 - accuracy: 0.7488 - val_loss: 0.6036 - val_accuracy: 0.7391
Epoch 8/100


Epoch 59/100
31/31 [==============================] - 0s 3ms/step - loss: 0.3371 - accuracy: 0.9379 - val_loss: 0.3188 - val_accuracy: 0.9565
Epoch 60/100
31/31 [==============================] - 0s 2ms/step - loss: 0.3194 - accuracy: 0.9448 - val_loss: 0.3132 - val_accuracy: 0.9565
Epoch 61/100
31/31 [==============================] - 0s 2ms/step - loss: 0.3402 - accuracy: 0.9121 - val_loss: 0.3172 - val_accuracy: 0.9565
Epoch 62/100
31/31 [==============================] - 0s 2ms/step - loss: 0.3141 - accuracy: 0.9318 - val_loss: 0.3166 - val_accuracy: 0.9517
Epoch 63/100
31/31 [==============================] - 0s 2ms/step - loss: 0.3017 - accuracy: 0.9175 - val_loss: 0.3181 - val_accuracy: 0.9517
Epoch 64/100
31/31 [==============================] - 0s 3ms/step - loss: 0.3361 - accuracy: 0.9138 - val_loss: 0.3302 - val_accuracy: 0.9541
Epoch 65/100
31/31 [==============================] - 0s 3ms/step - loss: 0.3350 - accuracy: 0.9340 - val_loss: 0.3244 - val_accuracy: 0.9517
Epoch 

In [64]:
loss, accuracy = model.evaluate(test_data, test_labels)

19/19 [==============================] - 0s 735us/step - loss: 0.3156 - accuracy: 0.9628


In [65]:
loss

0.31560418009757996

In [66]:
accuracy

0.962774932384491

In [67]:
x = model.predict(test_data).astype(dtype="u8")

In [68]:
a=x

In [69]:
b= np.array(test_labels)

In [70]:
a= a.reshape(1,a.shape[0])

In [71]:
np.size(a != b) - np.count_nonzero(a == b)

148

In [72]:
con = tf.math.confusion_matrix(labels= test_labels, predictions=x )

In [73]:
con

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[443,   0],
       [148,   0]])>